In [35]:
from casPYan import Node, Edge, connect, step, apply_spike
import casPYan as cap
import tabulate
import json
import copy


from IPython.core.debugger import Pdb

In [36]:
def network_table(nodes):
    # Show network output/state
    counts = cap.fires(nodes)
    vectors = cap.vectors(nodes)
    lastfires = cap.lastfires(nodes)
    charges = cap.charges(nodes)
    # histories = [node.history for node in nodes]

    data = zip(
        range(len(counts)),
        charges,
        counts,
        lastfires,
        vectors,
        # histories,
    )
    data = [[
        'id',
        'charge',
        'fires',
        't_lastfire',
        't_fires',
        # 'histories',
    ]] + list(data)
    return tabulate.tabulate(data, tablefmt='html')

In [37]:
nodes = []  # create nodes object
inputs = i0, i1 = Node(1), Node(1)  # create true input nodes
outputs = out = [Node(1) for i in range(4)]  # create true output nodes

nodes += inputs  # add inputs and outputs to the list of all nodes
nodes += outputs

In [38]:
# apply_spike(in0, 1.0, 0)
def apply_spike_const(node, a, n):
    node.intake += [(a, delay) for delay in range(n)]

def read_jsonfile(path):
    with open(path, 'r') as f:
        return json.loads(f.read())

nets = [cap.network_from_json(read_jsonfile(f"networks/b2_{i}.json")) for i in range(10 + 1)]


In [39]:
def foldin_subnet(subnet, output, invert=False):
    global nodes
    subnodes, subinputs, suboutputs = copy.deepcopy(subnet)
    nodes += list(subnodes.values())
    connect(inputs[1 if invert else 0], subinputs[0], weight=255)
    connect(inputs[0 if invert else 1], subinputs[1], weight=255)
    connect(suboutputs[0], output, weight=255)

# -v +v -w +w
#  0  1  2  3

# see stuff
foldin_subnet(nets[10], out[1])  # +v
foldin_subnet(nets[1], out[3])  # +w
# don't see stuff
foldin_subnet(nets[10], out[1], invert=1)  # +v
foldin_subnet(nets[10], out[2], invert=1)  # -w

In [40]:
# cap.step(nodes)
# inputs[1].callback_prestep_fire = lambda node: Pdb().set_trace()

In [41]:
apply_spike_const(inputs[0], 255, 10)

In [42]:
cap.run(nodes, 5)
cap.run(nodes, 10)
# cap.run(nodes, 0)

In [43]:
network_table(nodes)

id,charge,fires,t_lastfire,t_fires
0,0,6,5,"[0, 1, 2, 3, 4, 5]"
1,0,0,-1.0,[]
2,0,0,-1.0,[]
3,0,10,9,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]"
4,0,10,9,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]"
5,0,0,-1.0,[]
6,0.0,7,6,"[0, 1, 2, 3, 4, 5, 6]"
7,0.0,0,-1.0,[]
8,0.0,0,-1.0,[]
9,0,0,-1.0,[]


In [44]:


# nodes, inputs, outputs = cap.network_from_json(j)
nodes

[Node at 7f61fc19cb90 w/ Threshold: 1, Delay: None, Leak: None, children: ['f080', '7410', '6f60', '5670'],
 Node at 7f61fc0c7980 w/ Threshold: 1, Delay: None, Leak: None, children: ['ef00', '7c20', '6cf0', '5c10'],
 Node at 7f6200782db0 w/ Threshold: 1, Delay: None, Leak: None, children: [],
 Node at 7f61fc0c4fe0 w/ Threshold: 1, Delay: None, Leak: None, children: [],
 Node at 7f61fc14de80 w/ Threshold: 1, Delay: None, Leak: None, children: [],
 Node at 7f61fc0d5160 w/ Threshold: 1, Delay: None, Leak: None, children: [],
 Node at 7f61fc19f080 w/ Threshold: 0.0, Delay: 0.0, Leak: 0.0, children: [],
 Node at 7f61fc19ef00 w/ Threshold: 0.0, Delay: 0.0, Leak: 1.0, children: ['4dd0'],
 Node at 7f61fc0d4dd0 w/ Threshold: 0.0, Delay: 0.0, Leak: 3.0, children: ['f080', 'ef00', '4fe0'],
 Node at 7f61fc0c57f0 w/ Threshold: 68.0, Delay: 0.0, Leak: 2.0, children: [],
 Node at 7f61fc0d7410 w/ Threshold: 125.0, Delay: 0.0, Leak: 3.0, children: [],
 Node at 7f61fc0d7c20 w/ Threshold: 14.0, Delay: 0.

In [45]:
d = cap.to_tennlab(nodes, inputs, outputs)
d['Associated_Data']['application'].update({'proc_ticks': 10})
j = json.dumps(d)
with open('networks/combonet_vv_1.json', 'w') as f:
    f.write(j)
d

{'Associated_Data': {'application': {'proc_ticks': 10},
  'label': None,
  'processor': {'Leak_Enable': True,
   'Max_Leak': 4,
   'Min_Leak': -1,
   'Max_Weight': 127,
   'Min_Weight': -127,
   'Max_Threshold': 127,
   'Min_Threshold': 0,
   'Max_Synapse_Delay': 255,
   'Min_Synapse_Delay': 0,
   'Max_Axon_Delay': 0,
   'Min_Axon_Delay': 0}},
 'Properties': {'network_properties': [],
  'node_properties': [{'index': 0,
    'name': 'Threshold',
    'max_value': 127.0,
    'min_value': 0.0,
    'size': 1,
    'type': 73},
   {'index': 1,
    'name': 'Leak',
    'max_value': 4.0,
    'min_value': -1.0,
    'size': 1,
    'type': 73},
   {'index': 2,
    'name': 'Delay',
    'max_value': 0.0,
    'min_value': 0.0,
    'size': 1,
    'type': 73}],
  'edge_properties': [{'index': 0,
    'name': 'Weight',
    'max_value': 127.0,
    'min_value': -127.0,
    'size': 1,
    'type': 73},
   {'index': 1,
    'name': 'Delay',
    'max_value': 300.0,
    'min_value': 0.0,
    'size': 1,
    'type':

In [46]:
nodes, inputs, outputs = cap.network_from_json(read_jsonfile(f"networks/combonet_vv_1.json"))
nodes = nodes.values()

In [47]:
apply_spike_const(inputs[1], 255, 10)

In [48]:
cap.run(nodes, 5)
cap.run(nodes, 10)
# cap.run(nodes, 0)

In [49]:
network_table(nodes)

id,charge,fires,t_lastfire,t_fires
0,0,0,-1.0,[]
1,0,6,5,"[0, 1, 2, 3, 4, 5]"
2,0,0,-1.0,[]
3,0,10,9,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]"
4,0,0,-1.0,[]
5,255.0,1,6,[6]
6,0.0,0,-1.0,[]
7,0.0,7,6,"[0, 1, 2, 3, 4, 5, 6]"
8,0.0,9,8,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"
9,0,0,-1.0,[]
